In [9]:
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from lstm_pipe import *

In [42]:
file_path = '../output1/merge.pickle'
with open(file_path, 'rb') as f:
    obj = pkl.load(f)
    
df = pd.DataFrame(obj)
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 754599 entries, (Timestamp('2010-01-31 00:00:00'), 10001) to (Timestamp('2023-12-31 00:00:00'), 93436)
Data columns (total 222 columns):
 #    Column                Non-Null Count   Dtype         
---   ------                --------------   -----         
 0    gvkey                 744562 non-null  string        
 1    datadate              689795 non-null  datetime64[ns]
 2    primary               754599 non-null  bool          
 3    exchcd                754599 non-null  float64       
 4    me                    741780 non-null  float64       
 5    ret                   740651 non-null  float64       
 6    exret                 740651 non-null  float64       
 7    rf                    754599 non-null  float64       
 8    age                   754599 non-null  float64       
 9    mispricing_mgmt       619898 non-null  float64       
 10   mispricing_perf       694334 non-null  float64       
 11   qmj                   30694

In [13]:
def impute_permno(df):
    """Imputes missing values within each permno group using the median.

    Parameters:
    -----------
    df: A Pandas DataFrame with a MultiIndex.

    Returns:
    --------
    df_imputed: A DataFrame with imputed values.
    """
    print('USING NEW IMPUTE METHOD')

    # Remove columns with high NaN percentage
    print('Removing columns with a NaN % > 30. . .\n')
    for col in tqdm(df.columns):
        percent_NaN = df[col].isna().mean()
        if percent_NaN > 0.3:
            # print(f'\n Deleting column {col} with NaN: {percent_NaN}%. . .\n')
            df.drop(col, axis=1, inplace=True)

    print('Imputing median for each permno. . .\n')
    # Get unique permno IDs
    permno_ids = df.index.get_level_values('permno').unique()

    # Iterate over permno IDs and impute missing values
    for permno in tqdm(permno_ids):
        df.loc[df.index.get_level_values('permno') == permno] = df.loc[
            df.index.get_level_values('permno') == permno
        ].fillna(df.loc[df.index.get_level_values('permno') == permno].median())

    return df

In [ ]:
# Impute data, median of each permno group for NaN vals
non_vals=["gvkey", "datadate", "primary"] #, "exchcd", "ret", "exret", "rf", "me"]
non_vals_temp = df[non_vals]
dropped_df = df.drop(non_vals,axis=1)
print('Imputation of NaN values. . .\n')
imputed_df = impute_permno(dropped_df)
del dropped_df

Imputation of NaN values. . .

USING NEW IMPUTE METHOD
Removing columns with a NaN % > 30. . .



100%|██████████| 214/214 [00:29<00:00,  7.15it/s]


Imputing median for each permno. . .



100%|██████████| 9206/9206 [28:52<00:00,  5.31it/s] 


In [30]:
from sklearn.preprocessing import MinMaxScaler
print('Normalisation of vals. . .\n')
# Normalize features
or_cols = imputed_df.columns
scaler = MinMaxScaler()
temp_transformed_df = scaler.fit_transform(imputed_df)
transformed_df = pd.DataFrame(temp_transformed_df, index=imputed_df.index, columns=or_cols)

Normalisation of vals. . .



In [43]:
# Scaler converts to np.ndarray, so convert back to pandas.DF
# concat_df = pd.DataFrame(transformed_df, columns = or_cols)

df = pd.concat([non_vals_temp, transformed_df], axis=1)
# pd.concat()

In [41]:
non_vals_temp.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 754599 entries, (Timestamp('2010-01-31 00:00:00'), 10001) to (Timestamp('2023-12-31 00:00:00'), 93436)
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   gvkey     744562 non-null  string        
 1   datadate  689795 non-null  datetime64[ns]
 2   primary   754599 non-null  bool          
 3   exchcd    754599 non-null  float64       
 4   ret       740651 non-null  float64       
 5   exret     740651 non-null  float64       
 6   rf        754599 non-null  float64       
 7   me        741780 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(5), string(1)
memory usage: 44.0 MB


In [44]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 754599 entries, (Timestamp('2010-01-31 00:00:00'), 10001) to (Timestamp('2023-12-31 00:00:00'), 93436)
Data columns (total 163 columns):
 #    Column             Non-Null Count   Dtype         
---   ------             --------------   -----         
 0    gvkey              744562 non-null  string        
 1    datadate           689795 non-null  datetime64[ns]
 2    primary            754599 non-null  bool          
 3    exchcd             754599 non-null  float64       
 4    ret                740651 non-null  float64       
 5    exret              740651 non-null  float64       
 6    rf                 754599 non-null  float64       
 7    me                 741780 non-null  float64       
 8    age                754599 non-null  float64       
 9    mispricing_mgmt    728665 non-null  float64       
 10   mispricing_perf    752104 non-null  float64       
 11   qmj_prof           731419 non-null  float64       
 12   qmj_safet

In [45]:
# Get indexes
dates = df.index.get_level_values('date')
permno_ids = df.index.get_level_values('permno')
dates[0:10]

DatetimeIndex(['2010-01-31', '2010-02-28', '2010-03-31', '2010-04-30',
               '2010-05-31', '2010-06-30', '2010-07-31', '2010-08-31',
               '2010-09-30', '2010-10-31'],
              dtype='datetime64[ns]', name='date', freq=None)

In [4]:
# Count unique permnos
count = 0
permno_set = set()
for p in permno_ids.to_list():
    if p not in permno_set:
        count += 1
        permno_set.add(p)
count

9206

In [5]:
exclude_columns = ["gvkey", "datadate", "primary", "exchcd", "ret", "exret", "rf", "me"]
exret = df.exret
features = df.drop(columns=exclude_columns)
exret.head()

date        permno
2010-01-31  10001    -0.018960
2010-02-28  10001    -0.000666
2010-03-31  10001     0.020575
2010-04-30  10001     0.124274
2010-05-31  10001     0.004717
Name: exret, dtype: float64

In [9]:
p0 = permno_set.pop()
idxs = [idx for idx, val in enumerate(permno_ids) if val == p]
temp_df = features.iloc[idxs]
temp_exret = exret.iloc[idxs]
i=0
x_temp = temp_df.iloc[i:i+5]
y_temp = temp_exret.iloc[i+5:i+5+1]

In [7]:
# Cross-section on permno 10001
df_10001 = df.xs(10001, level='permno')

# Filter dates before '2015-01-01' (exclusive)
filtered_df = df_10001[df_10001.index < '2015-01-01']

In [10]:
filtered_df

,gvkey,datadate,primary,exchcd,me,ret,exret,rf,age,mispricing_mgmt,...,niq_at,niq_at_chg1,niq_be,niq_be_chg1,niq_su,ocfq_saleq_std,roavol,rsup,saleq_su,stdacc
date,,,,,,,,,,,,,,,,,,,,,
2010-01-31,012994,NaT,True,2.0,43.871660,-0.018932,-0.018960,0.000028,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28,012994,NaT,True,2.0,43.646632,-0.000656,-0.000666,0.000010,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,012994,NaT,True,2.0,44.351370,0.020643,0.020575,0.000068,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,012994,NaT,True,2.0,69.137300,0.124385,0.124274,0.000111,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31,012994,NaT,True,2.0,69.209400,0.004829,0.004717,0.000112,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,012994,NaT,True,2.0,66.028800,-0.043421,-0.043548,0.000127,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-31,012994,2010-03-31,True,2.0,71.257600,0.083333,0.083191,0.000142,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-31,012994,2010-03-31,True,2.0,62.983083,-0.111263,-0.111389,0.000126,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30,012994,2010-03-31,True,2.0,67.531760,0.076560,0.076435,0.000125,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
x_temp.to_numpy()

array([[ 9.40000000e+01,  1.66704276e-01,  5.34643530e-01, ...,
         2.51832431e-02,  1.34928009e+00,  1.09124034e-01],
       [ 9.50000000e+01,  1.59252012e-01,  4.50969071e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.60000000e+01,  1.59466296e-01,  4.84702059e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.70000000e+01,  1.67951882e-01,  4.59847269e-01, ...,
         1.19740330e-02, -5.46508811e-01,  1.09631669e-01],
       [ 9.80000000e+01,  3.30914054e-01,  3.58832925e-01, ...,
         1.90965471e-02,  8.64726497e-02,  1.21669637e-01]])

In [36]:
x_temp

,,age,mispricing_mgmt,mispricing_perf,qmj,qmj_growth,qmj_prof,qmj_safety,beta_60m,chcsho_12m,chmom,...,niq_at,niq_at_chg1,niq_be,niq_be_chg1,niq_su,ocfq_saleq_std,roavol,rsup,saleq_su,stdacc
date,permno,,,,,,,,,,,,,,,,,,,,,
2017-12-31,93436,94.0,0.166704,0.534644,-0.664615,1.093642,-1.208096,-0.708078,1.199302,0.044788,-0.831205,...,-0.013427,0.018470,-0.067445,0.234697,0.187433,0.152166,0.011885,0.025183,1.349280,0.109124
2018-01-31,93436,95.0,0.159252,0.450969,-1.041790,0.694345,-1.291706,-0.951286,1.226342,0.044084,-0.188624,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-02-28,93436,96.0,0.159466,0.484702,-1.047236,0.700880,-1.291295,-0.945867,1.240570,0.044844,-0.459733,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-03-31,93436,97.0,0.167952,0.459847,-1.058551,0.698263,-1.286568,-1.004619,1.379444,0.034027,-0.445443,...,-0.023782,-0.025625,-0.121309,-0.129990,-2.937532,0.136547,0.011907,0.011974,-0.546509,0.109632
2018-04-30,93436,98.0,0.330914,0.358833,-0.469646,1.646153,-1.008476,-1.097450,1.342683,0.033690,-0.169096,...,-0.024028,-0.014392,-0.143341,-0.098075,-1.533735,0.130207,0.011355,0.019097,0.086473,0.121670


In [39]:
# Build data for lstm
past = 5 # 5 months of data
future = 1 # next month risk prem
X_arr,y = [],[]

for p in tqdm(permno_set):
    idxs = [idx for idx, val in enumerate(permno_ids) if val == p]
    temp_df = df.iloc[idxs].to_numpy()
    temp_exret = exret.iloc[idxs].to_numpy()
    
    for i in range(len(temp_df) - past - future + 1):
        X_arr.append(temp_df[i:i+past])
        y.append(temp_exret[i+past:i+past+future])
        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9205/9205 [21:37<00:00,  7.22it/s]
